Segmenting and Clustering Neighborhoods in Toronto:

The project includes scraping the Wikipedia page - 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
#read wikipedia page for toronto neighborhoods
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [3]:
soup = BeautifulSoup(page.text,'html.parser')

In [4]:
table = soup.find('table').find_all('td')

In [5]:
#Search all the postcode, borough, neighborhood 
table_contents = []
for i in range(0,len(table)):
    cell = {}
    cell['PostalCode'] = table[i].contents[1].text[:3]
    if table[i].contents[1].contents[2].text == 'Not assigned':
        cell['Borough'] = 'Not assigned'
        cell['Neighborhood'] = 'Not assigned'
    else:
        cell['Borough'] = table[i].contents[1].contents[2].text.split('(')[0]
        cell['Neighborhood'] = table[i].contents[1].contents[2].text.split('(')[1][:-1].replace(' /',',')
        table_contents.append(cell)  

In [6]:
import pandas as pd

In [7]:
# Create dataframe from above list
df=pd.DataFrame(table_contents)

In [8]:
# Remove all the 'not assigned' boroughs from the dataframe 
df=df[df.Borough != 'Not assigned']
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [9]:
df.shape

(103, 3)

In [10]:
# install geocoder in anaconda prompt - 'pip install geocoder'
import geocoder 

In [11]:
def get_geocode(postal_code):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

Tried geocoder, but runtime is very high

In [12]:
# read csv file
geocode_df = pd.read_csv('https://cocl.us/Geospatial_data')

In [13]:
geocode_df.rename(columns = {'Postal Code':'PostalCode'},inplace = True)

In [14]:
# joining two dataframes using pandas
result = pd.merge(df,geocode_df,on='PostalCode')

In [15]:
result

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
